<a href="https://colab.research.google.com/github/robert-myers/myanimelist-recommender/blob/master/notebooks/SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install surprise

     |████████████████████████████████| 6.5MB 2.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678244 sha256=b24ed5c7936a70ab9ccbbbe6ac43aa1a87d065fcec0b8093fadad50a5dc63c6e
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import numpy as np
import pandas as pd
import random

from surprise import accuracy, Dataset, SVD, SVDpp, NMF, Reader
from surprise.dump import dump
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

my_seed = 8182868
random.seed(my_seed)
np.random.seed(my_seed)

In [3]:
%%time
# Creation of the dataframe.
df = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/surprise/uid_iid_rating.zip")

CPU times: user 11.4 s, sys: 1.42 s, total: 12.9 s
Wall time: 24.5 s


In [4]:
%%time
# A reader is needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['username', 'anime_id', 'my_score']], reader)

# sample random trainset and testset
# test set is made of 1% of the ratings.
trainset, testset = train_test_split(data, train_size=.99, test_size=.01)

CPU times: user 2min 46s, sys: 5.93 s, total: 2min 52s
Wall time: 2min 52s


In [0]:
# %%time
# init_mean = df["my_score"].mean()
# init_std_dev = df["my_score"].std()

# param_grid = {
#     'init_mean': [0, init_mean],
#     'init_std_dev': [0.1, init_std_dev],
# }

# gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=2)

# gs.fit(data)

# # best RMSE score
# print(gs.best_score['rmse'])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params['rmse'])

In [0]:
%%time
algo = SVD()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

In [0]:
dump("svd.pickle", predictions=predictions, algo=algo, verbose=1)